# Datomic & Datalog Demo

This notebook demonstrates Datomic with Datalog queries to teach Prolog concepts.

## Prolog Connections
- **Logical Variables**: `?name`, `?course`  
- **Declarative Style**: Describe what you want, not how to get it
- **Rule-Based Logic**: Define relationships and let the engine find solutions
- **Unification**: Pattern matching across data relationships

In [ ]:
;; First, let's load the Datomic dependencies
(require '[clojupyter.display :as display])
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[com.d/peer "1.0.7387"])
(require '[datomic.api :as d])

## 1. Create In-Memory Database

In [ ]:
;; 1. Define the database URI
(def db-uri "datomic:mem://student-courses")

;; 2. Create the database (idempotent operation)
(d/create-database db-uri)

;; 3. Now, connect to it
(def conn (d/connect db-uri))
(println "✅ Connected to in-memory database")

## 2. Define Schema

In [ ]:
;; Define our data structure
(def schema-tx
  [{:db/ident :student/name
    :db/valueType :db.type/string
    :db/cardinality :db.cardinality/one
    :db/doc "A student's name"}
   {:db/ident :student/course
    :db/valueType :db.type/string
    :db/cardinality :db.cardinality/many
    :db/doc "A course the student is taking"}])

;; Transact the schema to the database
@(d/transact conn schema-tx)
(println "✅ Schema defined")

## 3. Add Sample Data

In [ ]:
;; Add some students and their courses
(def sample-data-tx
  [{:student/name "Alice"
    :student/course #{"Logic 101" "Databases"}}
   {:student/name "Bob" 
    :student/course #{"Logic 101" "Prolog"}}
   {:student/name "Charlie"
    :student/course #{"Databases" "AI"}}
   {:student/name "Diana"
    :student/course #{"Prolog" "AI"}}])

@(d/transact conn sample-data-tx)
(println "✅ Sample data added")

## 4. Basic Datalog Queries (Prolog-like)

In [ ]:
;; Query 1: Find all student names
;; Similar to Prolog: student_name(Name)
(println "All student names:")
(d/q '[:find ?name
             :where [?e :student/name ?name]]
           (d/db conn))

In [ ]:
;; Query 2: Find students taking "Logic 101"
;; Similar to Prolog: takes_course(Name, 'Logic 101')
(println "Students taking Logic 101:")
(d/q '[:find ?name
             :where [?e :student/name ?name]
                    [?e :student/course "Logic 101"]]
           (d/db conn))

In [ ]:
;; Query 3: Find students taking Prolog
(println "Students taking Prolog:")
(d/q '[:find ?name
             :where [?e :student/name ?name]
                    [?e :student/course "Prolog"]]
           (d/db conn))

## 5. Advanced Queries (Joins and Logic)

In [ ]:
;; Query 4: Find students who share courses
;; This demonstrates multi-variable logic like Prolog
(println "Students who share courses (pairs):")
(d/q '[:find ?name1 ?name2 ?shared-course
             :where [?e1 :student/name ?name1]
                    [?e2 :student/name ?name2]
                    [?e1 :student/course ?shared-course]
                    [?e2 :student/course ?shared-course]
                    [(not= ?e1 ?e2)]
                    [(< (compare ?name1 ?name2) 0)]]
           (d/db conn))

In [ ]:
;; Query 5: Find courses with multiple students
;; Shows aggregation and grouping
(println "Courses with enrollment counts:")
(d/q '[:find ?course (count ?name)
             :where [?e :student/name ?name]
                    [?e :student/course ?course]]
           (d/db conn))

## 6. Prolog-style Rules

In [ ]:
;; Define Datalog rules (similar to Prolog rules)
(def rules
  '[[(classmates ?name1 ?name2)
     [?e1 :student/name ?name1]
     [?e2 :student/name ?name2]
     [?e1 :student/course ?course]
     [?e2 :student/course ?course]
     [(not= ?e1 ?e2)]]])

;; Use the rule to find all classmates
(println "All classmates (using rules):")
(d/q '[:find ?name1 ?name2
             :in $ %
             :where (classmates ?name1 ?name2)]
           (d/db conn) rules)

## 7. Interactive Exploration

In [ ]:
;; Students can modify this query to explore the data
;; Try changing the course name or adding conditions
(println "Custom query - modify me!")
(d/q '[:find ?name ?course
             :where [?e :student/name ?name]
                    [?e :student/course ?course]
                    [(clojure.string/includes? ?course "AI")]]
           (d/db conn))

## Summary

### Prolog Concepts Demonstrated:
- **Logical Variables**: `?name`, `?course`, `?name1`, `?name2`
- **Facts**: Student-course relationships as database facts
- **Queries**: Pattern matching across relationships
- **Rules**: Defining relationship patterns (like Prolog predicates)
- **Unification**: Joining data across multiple variables
- **Declarative Programming**: Describing what you want, not how to compute it

This demonstrates how Datalog brings Prolog-style logic programming to database queries!